# Example Generation

In [1]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/sygus
Root directory: /home/ubuntu/arga-arc


In [2]:
from sygus.utils import sample_gpt_solutions, SygusProblem, SygusBenchmark, cleanup_completion, add_sygus_prefix, is_comment, is_synth_fun, get_synth_fun, get_constraints, ModelName, get_function_definitions, OPENAI_MODEL_NAMES, OPENAI, TOGETHER_MODEL_NAMES, TOGETHER

Current directory: /home/ubuntu/arga-arc/sygus
Root directory: /home/ubuntu/arga-arc


In [3]:
import typing as t
from pprint import pprint
from dataclasses import dataclass
import sexpdata as sexp
from sexpdata import Symbol
import json
import random
import math
import re
import datetime

In [4]:
BENCHMARKS_DIRECTORY = ROOT_DIRECTORY / "sygus/Probe/src/test/benchmarks"
CIRCUIT_DIRECTORY = BENCHMARKS_DIRECTORY / "circuit/test"
HACKERS_DELIGHT_DIRECTORY = BENCHMARKS_DIRECTORY / "hackers-delight"
LARGER_STRING_GRAMMAR_DIRECTORY = BENCHMARKS_DIRECTORY / "larger-grammar"
STRING_GRAMMAR_DIRECTORY = BENCHMARKS_DIRECTORY / "string"

## Hackers Delight

In [5]:
HACKERS_DELIGHT_PROBLEMS = {}
HACKERS_DELIGHT_COMMENTS = {}

for file in HACKERS_DELIGHT_DIRECTORY.iterdir():
    print(file.name)
    with open(file, "r") as f:
        contents = file.read_text()
        HACKERS_DELIGHT_PROBLEMS[file.name] = sexp.loads(contents)
        HACKERS_DELIGHT_COMMENTS[file.name] = [
            line for line in contents.split("\n") if is_comment(line)
        ]


hd-25.sl
hd-24.sl
hd-09.sl
hd-26.sl
hd-27.sl
hd-08.sl
hd-23.sl
hd-22.sl
hd-18.sl
hd-20.sl
hd-21.sl
hd-19.sl
hd-07.sl
hd-10.sl
hd-11.sl
hd-06.sl
hd-04.sl
hd-13.sl
hd-12.sl
hd-05.sl
hd-16.sl
hd-01.sl
hd-17.sl
hd-15.sl
hd-02.sl
hd-03.sl
hd-14.sl


In [6]:
HACKERS_DELIGHT_SYNTH_FUNS = {}
HACKERS_DELIGHT_CONSTRAINTS = {}

for filename, sexps in HACKERS_DELIGHT_PROBLEMS.items():
    synth_fun = get_synth_fun(sexps)
    HACKERS_DELIGHT_SYNTH_FUNS[filename] = synth_fun
    HACKERS_DELIGHT_CONSTRAINTS[filename] = get_constraints(sexps)

pprint(HACKERS_DELIGHT_SYNTH_FUNS["hd-25.sl"])
pprint(HACKERS_DELIGHT_CONSTRAINTS["hd-25.sl"])

[Symbol('synth-fun'),
 Symbol('f'),
 [[Symbol('x'), [Symbol('BitVec'), 64]], [Symbol('y'), [Symbol('BitVec'), 64]]],
 [Symbol('BitVec'), 64],
 [[Symbol('Start'),
   [Symbol('BitVec'), 64],
   [[Symbol('bvnot'), Symbol('Start')],
    [Symbol('bvxor'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvand'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvor'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvneg'), Symbol('Start')],
    [Symbol('bvadd'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvmul'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvudiv'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvurem'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvlshr'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvashr'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvshl'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvsdiv'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvsrem'), Symbol('Start'), Symbol('Start')],
    [Symbol('bvsub'), Symbol('Start'), Sym

In [7]:
HACKERS_DELIGHT_FUNCTION_DEFINITIONS = {}

for filename, sexps in HACKERS_DELIGHT_PROBLEMS.items():
    HACKERS_DELIGHT_FUNCTION_DEFINITIONS[filename] = get_function_definitions(sexps)

pprint(HACKERS_DELIGHT_FUNCTION_DEFINITIONS["hd-25.sl"])

[[Symbol('define-fun'),
  Symbol('hd25'),
  [[Symbol('x'), [Symbol('BitVec'), 64]],
   [Symbol('y'), [Symbol('BitVec'), 64]]],
  [Symbol('BitVec'), 64],
  [Symbol('bvadd'),
   [Symbol('bvadd'),
    [Symbol('bvlshr'),
     [Symbol('bvadd'),
      [Symbol('bvmul'),
       [Symbol('bvand'), Symbol('x'), Symbol('#xffffffffffffffff')],
       [Symbol('bvlshr'), Symbol('y'), Symbol('#x0000000000000010')]],
      [Symbol('bvand'),
       [Symbol('bvadd'),
        [Symbol('bvmul'),
         [Symbol('bvlshr'), Symbol('x'), Symbol('#x0000000000000010')],
         [Symbol('bvand'), Symbol('y'), Symbol('#xffffffffffffffff')]],
        [Symbol('bvlshr'),
         [Symbol('bvmul'),
          [Symbol('bvand'), Symbol('x'), Symbol('#xffffffffffffffff')],
          [Symbol('bvand'), Symbol('y'), Symbol('#xffffffffffffffff')]],
         Symbol('#x0000000000000010')]],
       Symbol('#xffffffffffffffff')]],
     Symbol('#x0000000000000010')],
    [Symbol('bvlshr'),
     [Symbol('bvadd'),
      [Symbol('b

In [8]:
EXAMPLE_GENERATION_SYSTEM_PROMPT = """You are a coding assistant. Be precise and terse.
You will be given a SyGus grammar, a natural language specification, a set of function definitions and a set of constraints.
Your task is to use the given information to generate a 10 of input-output examples that satisfy the constraints."""

TOOL = {
        "type": "function",
        "function": {
            "name": "send_examples",
            "description": "send a list of input-output examples to the user",
            "parameters": {
                "type": "object",
                "properties": {
                    "examples": {
                        "type": "array",
                        "description": "a list of input-output examples",
                        "items": {
                            "type": "object",
                            "description": "a single input-output example",
                            "properties": {
                                "inputs": {
                                    "type": "array",
                                    "description": "a list of inputs",
                                    "items": {
                                        "type": "string",
                                        "description": "a single input"
                                    }
                                },
                                "output": {
                                    "type": "string",
                                    "description": "the output"
                                }
                            }
                        }
                    }
                }
            }
        }
    }

In [9]:
TEST_GRAMMAR = HACKERS_DELIGHT_SYNTH_FUNS["hd-25.sl"]
TEST_NAT_LANG_SPEC = "\n".join(HACKERS_DELIGHT_COMMENTS["hd-25.sl"])
TEST_FUNCTION_DEFINITIONS = "\n".join([sexp.dumps(fd) for fd in HACKERS_DELIGHT_FUNCTION_DEFINITIONS["hd-25.sl"]])
TEST_CONSTRAINTS = "\n".join([sexp.dumps(c) for c in  HACKERS_DELIGHT_CONSTRAINTS["hd-25.sl"]])

TEST_USER_MESSAGE = f"""[GRAMMAR]
{sexp.dumps(TEST_GRAMMAR)}

[NATURAL LANGUAGE SPECIFICATION]
{TEST_NAT_LANG_SPEC}

[FUNCTION DEFINITIONS]
{TEST_FUNCTION_DEFINITIONS}

[CONSTRAINTS]
{TEST_CONSTRAINTS}"""

In [10]:
pprint(TEST_USER_MESSAGE)

('[GRAMMAR]\n'
 '(synth-fun f ((x (BitVec 64)) (y (BitVec 64))) (BitVec 64) ((Start (BitVec '
 '64) ((bvnot Start) (bvxor Start Start) (bvand Start Start) (bvor Start '
 'Start) (bvneg Start) (bvadd Start Start) (bvmul Start Start) (bvudiv Start '
 'Start) (bvurem Start Start) (bvlshr Start Start) (bvashr Start Start) (bvshl '
 'Start Start) (bvsdiv Start Start) (bvsrem Start Start) (bvsub Start Start) x '
 'y \\#x0000000000000001 \\#x0000000000000010 \\#xffffffffffffffff (ite '
 'StartBool Start Start))) (StartBool Bool ((= Start Start)))))\n'
 '\n'
 '[NATURAL LANGUAGE SPECIFICATION]\n'
 '; Compute higher order half of product of x and y.\n'
 '\n'
 '[FUNCTION DEFINITIONS]\n'
 '(define-fun hd25 ((x (BitVec 64)) (y (BitVec 64))) (BitVec 64) (bvadd (bvadd '
 '(bvlshr (bvadd (bvmul (bvand x \\#xffffffffffffffff) (bvlshr y '
 '\\#x0000000000000010)) (bvand (bvadd (bvmul (bvlshr x \\#x0000000000000010) '
 '(bvand y \\#xffffffffffffffff)) (bvlshr (bvmul (bvand x '
 '\\#xffffffffffffffff) (bv

In [11]:
def generate_examples(filename: str, n=5):
    synth_fun = HACKERS_DELIGHT_SYNTH_FUNS[filename]
    function_definitions = "\n".join([sexp.dumps(fd) for fd in HACKERS_DELIGHT_FUNCTION_DEFINITIONS[filename]])
    constraints = "\n".join([sexp.dumps(c) for c in  HACKERS_DELIGHT_CONSTRAINTS[filename]])
    natural_language_spec = "\n".join(HACKERS_DELIGHT_COMMENTS[filename])
    user_message = f"""[GRAMMAR]
{sexp.dumps(synth_fun)}

[NATURAL LANGUAGE SPECIFICATION]
{natural_language_spec}

[FUNCTION DEFINITIONS]
{function_definitions}

[CONSTRAINTS]
{constraints}"""
    completion = OPENAI.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": EXAMPLE_GENERATION_SYSTEM_PROMPT},
            {"role": "user", "content": user_message},
        ],
        tools=[TOOL],
        tool_choice=TOOL,
        n=n
    )
    examples = []
    for choice in completion.choices:
        try:
            arguments = json.loads(choice.message.tool_calls[0].function.arguments)
            examples.extend(arguments["examples"])
        except Exception as e:
            print(e)
            continue

In [ ]:
import os
import pickle

if os.path.exists("./HACKERS_DELIGHT_GENERATED_EXAMPLES.pkl"):
    with open("./HACKERS_DELIGHT_GENERATED_EXAMPLES.pkl", "r") as f:
        HACKERS_DELIGHT_GENERATED_EXAMPLES = pickle.load(f)
else:
    HACKERS_DELIGHT_GENERATED_EXAMPLES = {}
    for filename in HACKERS_DELIGHT_SYNTH_FUNS.keys():
        print(filename)
        HACKERS_DELIGHT_GENERATED_EXAMPLES[filename] = generate_examples(filename, n=5)
    with open("./HACKERS_DELIGHT_GENERATED_EXAMPLES.pkl", "w") as f:
        pickle.dump(HACKERS_DELIGHT_GENERATED_EXAMPLES, f)

## Circuit

In [12]:
CIRCUIT_PROBLEMS = {}
CIRCUIT_COMMENTS = {}

for file in CIRCUIT_DIRECTORY.iterdir():
    print(file.name)
    with open(file, "r") as f:
        contents = file.read_text()
        CIRCUIT_PROBLEMS[file.name] = sexp.loads(contents)
        CIRCUIT_COMMENTS[file.name] = [
            line for line in contents.split("\n") if is_comment(line)
        ]

CrCy_10-sbox2-D7-sIn1.sl
CrCy_6-P10-D7-sIn.sl
CrCy_6-P10-D7-sIn3.sl
CrCy_10-sbox2-D5-sIn104.sl
CrCy_9-pprmAll-D7-sIn1.sl
CrCy_6-P10-D5-sIn.sl
CrCy_10-sbox2-D5-sIn88.sl
CrCy_8-P12-D7-sIn5.sl
CrCy_10-sbox2-D5-sIn89.sl
CrCy_2-P6_2-P6.sl
CrCy_6-P10-D7-sIn5.sl
CrCy_8-P12-D7-sIn1.sl
CrCy_10-sbox2-D5-sIn79.sl
CrCy_10-sbox2-D5-sIn78.sl
CrCy_10-sbox2-D5-sIn91.sl
CrCy_6-P10-D9-sIn5.sl
CrCy_8-P12-D5-sIn1.sl
CrCy_8-P12-D9-sIn1.sl
CrCy_10-sbox2-D5-sIn90.sl
CrCy_10-sbox2-D5-sIn92.sl
CrCy_10-sbox2-D5-sIn15.sl
CrCy_8-P12-D5-sIn3.sl
CrCy_10-sbox2-D5-sIn14.sl
CrCy_10-sbox2-D5-sIn80.sl
CrCy_10-sbox2-D5-sIn77.sl
CrCy_6-P10-D9-sIn3.sl
CrCy_6-P10-D5-sIn3.sl
CrCy_9-pprmAll-D5-sIn1.sl
CrCy_9-pprmAll-D9-sIn1.sl
CrCy_10-sbox2-D5-sIn76.sl
CrCy_6-P10-D9-sIn.sl


In [13]:
CIRCUIT_SYNTH_FUNS = {}
CIRCUIT_CONSTRAINTS = {}

for filename, sexps in CIRCUIT_PROBLEMS.items():
    synth_fun = get_synth_fun(sexps)
    CIRCUIT_SYNTH_FUNS[filename] = synth_fun
    CIRCUIT_CONSTRAINTS[filename] = get_constraints(sexps)

pprint(CIRCUIT_SYNTH_FUNS["CrCy_10-sbox2-D7-sIn1.sl"])
pprint(CIRCUIT_CONSTRAINTS["CrCy_10-sbox2-D7-sIn1.sl"])

[Symbol('synth-fun'),
 Symbol('skel'),
 [[Symbol('LN17'), Symbol('Bool')],
  [Symbol('k7'), Symbol('Bool')],
  [Symbol('LN34'), Symbol('Bool')],
  [Symbol('LN40'), Symbol('Bool')],
  [Symbol('LN47'), Symbol('Bool')],
  [Symbol('LN56'), Symbol('Bool')],
  [Symbol('LN65'), Symbol('Bool')],
  [Symbol('LN75'), Symbol('Bool')]],
 Symbol('Bool'),
 [[Symbol('Start'),
   Symbol('Bool'),
   [[Symbol('and'), Symbol('depth1'), Symbol('depth1')],
    [Symbol('not'), Symbol('depth1')],
    [Symbol('or'), Symbol('depth1'), Symbol('depth1')],
    [Symbol('xor'), Symbol('depth1'), Symbol('depth1')]]],
  [Symbol('depth1'),
   Symbol('Bool'),
   [[Symbol('and'), Symbol('depth2'), Symbol('depth2')],
    [Symbol('not'), Symbol('depth2')],
    [Symbol('or'), Symbol('depth2'), Symbol('depth2')],
    [Symbol('xor'), Symbol('depth2'), Symbol('depth2')],
    Symbol('LN75')]],
  [Symbol('depth2'),
   Symbol('Bool'),
   [[Symbol('and'), Symbol('depth3'), Symbol('depth3')],
    [Symbol('not'), Symbol('depth3')],


In [14]:
CURCUIT_FUNCTION_DEFINITIONS = {}

for filename, sexps in CIRCUIT_PROBLEMS.items():
    CURCUIT_FUNCTION_DEFINITIONS[filename] = get_function_definitions(sexps)

pprint(CURCUIT_FUNCTION_DEFINITIONS["CrCy_10-sbox2-D7-sIn1.sl"])

[[Symbol('define-fun'),
  Symbol('origCir'),
  [[Symbol('LN17'), Symbol('Bool')],
   [Symbol('k7'), Symbol('Bool')],
   [Symbol('LN34'), Symbol('Bool')],
   [Symbol('LN40'), Symbol('Bool')],
   [Symbol('LN47'), Symbol('Bool')],
   [Symbol('LN56'), Symbol('Bool')],
   [Symbol('LN65'), Symbol('Bool')],
   [Symbol('LN75'), Symbol('Bool')]],
  Symbol('Bool'),
  [Symbol('xor'),
   [Symbol('xor'),
    [Symbol('xor'),
     [Symbol('xor'),
      [Symbol('and'),
       Symbol('LN40'),
       [Symbol('xor'),
        [Symbol('xor'), Symbol('k7'), Symbol('LN17')],
        Symbol('LN34')]],
      Symbol('LN56')],
     Symbol('LN65')],
    Symbol('LN47')],
   Symbol('LN75')]]]


In [16]:
def generate_circuit_examples(filename: str, n=5):
    synth_fun = CIRCUIT_SYNTH_FUNS[filename]
    function_definitions = "\n".join([sexp.dumps(fd) for fd in CURCUIT_FUNCTION_DEFINITIONS[filename]])
    constraints = "\n".join([sexp.dumps(c) for c in  CIRCUIT_CONSTRAINTS[filename]])
    natural_language_spec = "\n".join(CIRCUIT_COMMENTS[filename])
    user_message = f"""[GRAMMAR]
{sexp.dumps(synth_fun)}

[NATURAL LANGUAGE SPECIFICATION]
{natural_language_spec}

[FUNCTION DEFINITIONS]
{function_definitions}

[CONSTRAINTS]
{constraints}"""
    completion = OPENAI.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": EXAMPLE_GENERATION_SYSTEM_PROMPT},
            {"role": "user", "content": user_message},
        ],
        tools=[TOOL],
        tool_choice=TOOL,
        n=n
    )
    examples = []
    for choice in completion.choices:
        try:
            arguments = json.loads(choice.message.tool_calls[0].function.arguments)
            examples.extend(arguments["examples"])
        except Exception as e:
            print(e)
            continue
    return examples



In [ ]:
if os.path.exists("./CIRCUIT_GENERATED_EXAMPLES.pkl"):
    with open("./CIRCUIT_GENERATED_EXAMPLES.pkl", "r") as f:
        CIRCUIT_GENERATED_EXAMPLES = pickle.load(f)
else:
    CIRCUIT_GENERATED_EXAMPLES = {}
    for filename in CIRCUIT_SYNTH_FUNS.keys():
        print(filename)
        CIRCUIT_GENERATED_EXAMPLES[filename] = generate_circuit_examples(filename, n=5)
    with open("./CIRCUIT_GENERATED_EXAMPLES.pkl", "w") as f:
        pickle.dump(CIRCUIT_GENERATED_EXAMPLES, f)

In [ ]:
pprint(CIRCUIT_GENERATED_EXAMPLES)

In [ ]:
# write to json
CIRCUIT_GENERATED_EXAMPLES_FILE = ROOT_DIRECTORY / "sygus/circuit-generated-examples.json"
CIRCUIT_GENERATED_EXAMPLES_FILE.write_text(json.dumps(CIRCUIT_GENERATED_EXAMPLES, indent=2))